In [18]:
import galois
import numpy as np
import math
# from py_ecc.bn128 import curve_order

curve_order = 967

# Creating a finite field (Galois field) which is basically Fq where q = 31
GF967 = galois.GF(curve_order)
# print(GF.properties)

######################################

def process_array(array):
    (rows, columns) = array.shape
    for i in range(0, rows):
        for j in range(0, columns):
            if(math.modf(array[i][j])[0] == 0.0):
                if(array[i][j] < 0):
                    array[i][j] = GF967(0) - GF967(-(array[i][j]))
            else:
                raise Exception("Fractional values not allowed in the R1CS arrays over prime field", GF967)
    return GF967(array)

######################################

lagrange_x_values = GF967(np.array([1,2,3]))

# These are the default R1CS values of L, R, S and W.
# Later we can take user inputs to create custom R1CS
default_L = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,4]
]))

default_R = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0]
]))

S = np.array([
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [2,1,0,0,-1,0]
])

default_S = process_array(S)

default_W = GF967(np.array([1,199,3,4,9,16]))

###############################################

# def process_lagrange_polynomial_array(polynomial_array):
#     array_length = polynomial_array.shape[0]
#     is_negative = False
#     for i in range(0, array_length):
#         # check whether the number is fractional
#         if(math.modf(polynomial_array[i])[0] == 0.0):
#             # Check if numbers are negative, if yes, convert them to corresponding positive number in Fp
#             if polynomial_array[i] < 0:
#                 polynomial_array[i] = polynomial_array[i] % curve_order
#         else:  
#             # This is the case where we have fractional coefficients
#             (fractional_part, integer_part) = math.modf(polynomial_array[i])
#     return polynomial_array

def reverse_tuple(tuple):
    new_tuple = tuple[::-1]
    return new_tuple

def find_num_columns_in_array(arr):
    shape_tuple = arr.shape
    if(len(shape_tuple) == 2): 
        return shape_tuple[1]
    else: 
        return shape_tuple[0]
    
def find_num_points_for_lagrange(arr):
    shape_tuple = arr.shape
    return shape_tuple[0]

def convert_column_to_lagrange_polynomial(array, col_num: int):
    required_column = array[ :, col_num: (col_num + 1)].flatten()
    
    col_polynomial = galois.lagrange_poly(lagrange_x_values, required_column)
    print(col_polynomial)
    print(col_polynomial.coeffs)
    
    if(col_polynomial == 0):
        col_polynomial_array = GF967(np.zeros(find_num_points_for_lagrange(array), dtype = int))
    elif(col_polynomial.coeffs.size < find_num_points_for_lagrange(array)):
        num_zeros_required = find_num_points_for_lagrange(array) - np.asarray(col_polynomial.coeffs).shape[0]
        col_polynomial_array = GF967(np.append(np.zeros(num_zeros_required, dtype=int), (np.asarray(col_polynomial.coeffs))))
    else:
        col_polynomial_array = GF967(np.asarray(col_polynomial.coeffs))
    
    return col_polynomial_array

print(convert_column_to_lagrange_polynomial(default_S, 4))

# def convert_arrays_into_poly_arrays(array):
#     array = process_array(array)
#     num_columns = find_num_columns_in_array(array)
#     for i in range(num_columns):
#         if(i == 0):
#             first_arr = convert_column_to_lagrange_polynomial(array, i)
#             poly_arrays = np.array([first_arr])
#         else:
#             poly_arr = convert_column_to_lagrange_polynomial(array, i)
#             poly_arrays = np.append(poly_arrays, poly_arr)

#     return poly_arrays.reshape(reverse_tuple(array.shape)).transpose()

# def multiply_poly_arrays_with_witness(array):
#     return np.matmul(array, default_W)

# ###############################################

# default_L_poly_array = convert_arrays_into_poly_arrays(default_L)
# print(default_L_poly_array)

# print("***********")

# default_R_poly_array = convert_arrays_into_poly_arrays(default_R)
# print(default_R_poly_array)

# print("***********")

# default_S_poly_array = convert_arrays_into_poly_arrays(default_S)
# print(default_S_poly_array)

# print("***********")

###############################################

## example 3
## -6 in GF (the above defined galios field)
print(GF967(0) - GF967(6))
print(GF967(0) - GF967(1))

## example 4
## 5/2 in GF (the above defined galios field)
print(GF967(0) - (GF967(5) * GF967(2)**-1))



966x + 2
[966   2]
[  0 966   2]
961
966
481


In [ ]:
x = 53.5123
print(math.modf(53.5123))
print(math.modf(x))
print(math.modf(x)[0] == 0.0)
print(math.modf(x)[0] * 100)
x = math.modf(x)[0] * 100
print(int(x))

example_1 = -6.5 % 11
print(example_1) # 3.5
# Since, 3.5 == 35 x 10**-1
print((35%11 * pow(10, -1, 11))%11)

example_2 = (65%11 * pow(10, -1, 11))%11
print(-example_2 % 11)